In [10]:
import pandas as pd
import json
json_string = """
{
  "intents": [
  {"tag": "greetings",
   "patterns": ["Hii", "Hey", "Hello", "What's going on", "Whats's up", "How you doing?"],
   "responses": ["Hey!", "How are you", "What can i do for you", "Hello"]
   },
  {
    "tag": "goodbye",
    "patterns":["see you later","goodbye","I am leaving","bye","tata","ciao","see ya"],
    "responses":["Goodbye!","Cao","Sayōnara","See you later"]
  },
  {
    "tag": "Name",
    "patterns":["Name","What is your name?","what should i call you?","Apka naam kya hai","cuál es tu nombre"],
    "responses":["My name is R","U can call me R"]
  },
  {
    "tag": "slang",
    "patterns":["fuck","fucker","chutiya","Harami"],
    "responses":["tu chutiya","tu madarchod","Go FUCK YOUR SELF SOME WHERE ELSE"]
 },
 {
    "tag": "language",
    "patterns" :["What language do you speak?","language","What do u speak?"],
    "responses":["I speak English","I loved to speak English","English","Some time I like to speak Spanish"]
 },
   {"tag": "age",
    "patterns": ["How old are you", "What is your age", "Age?"],
    "responses": ["I am 9!", "9 years old"]  
   },

   {"tag": "work",
   "patterns": ["What do you do", "What you like", "What give you happiness"],
   "responses": ["I'll work for you", "I would like to help you", "By helping you"]
   }

]}"""
data = json.loads(json_string)

In [16]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

! pip install tensorflow==1.2.0
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
ERROR: Could not find a version that satisfies the requirement tensorflow==1.2.0 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==1.2.0


ModuleNotFoundError: No module named 'tensorflow'

Python pickle module is used for serializing and de-serializing a Python object structure.

NLTK (Natural Language Toolkit) Library is a suite that contains libraries and programs for statistical language processing

Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item

Punkt This tokenizer divides a text into a list of sentences, by using an unsupervised algorithm to build a model

Tensor It is a foundation library that can be used to create Deep Learning models directly or by using wrapper libraries that simplify the process built on top of TensorFlow.

Stochastic Gradient Descent (SGD) is a simple yet very efficient approach to fitting linear classifiers and regressors under convex loss functions such as (linear) Support Vector Machines and Logistic Regression

In [ ]:
  lemmatizer = WordNetLemmatizer()
  intents = data
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']


In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
      word_list=nltk.word_tokenize(pattern)
      words.extend(word_list)
      documents.append((word_list, intent['tag']))
      if intent['tag'] not in classes:
          classes.append(intent['tag'])



In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))


pickle.dump(words, open('words.pkl' , 'wb'))
pickle.dump(classes, open('classes.pkl' , 'wb'))

training = []
output_empty = [0]* len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])


random.shuffle(training)
training = np.array(training)


train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print('Done')

In [ ]:
from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()
intents =data

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')


def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)


def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results  =[[i, r] for i, r in enumerate(res) if r> ERROR_THRESHOLD]


    results.sort(key=lambda x: x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent': classes[r[0]],'probability':str(r[1])})
    return return_list

def get_responses(intents_list,intents_json):
   tag=intents_list[0]['intent']
   list_of_intents=intents_json['intents']
   for i in list_of_intents:
       if i in list_of_intents:
          if i['tag']==tag:
              result=random.choice(i['responses'])
              break
   return result

print("GO! BOT is running!")



In [9]:
from tkinter import *
chat=Tk()
chat.title("Chat_Bot")
chat.geometry("400x600")
#function
def pri():
    ic=str(store.get())
    message = ic
    ints = predict_class(message)
    res= get_responses(ints,intents)
    print(res)
#head
head=Label(chat,text="WELCOME TO THE R CHAT BOT",font=("Algerian",18,"bold"))
head.pack(side=TOP)
#button
frame=Frame(chat,bg="gray")
frame.pack(side="bottom",anchor="se")
but=Button(frame,text="Sent",command=pri)
but.pack(anchor="se",side="bottom")
#chat bar
storage=StringVar()

store=Entry(chat, textvariable=storage)
store.pack(anchor="sw",side="bottom")


chat.mainloop()


fhtg
